# Evaluation of the models

### Task breakdown:

#### 1. Load Benchmark data for each query.

#### 2. Load Ranked Documents for each query.

#### 3. Evaluate the data

#### 4. Compare the models from the results obtained


### Testing code for evaluating model for 1 query with 1 model's result for that query

For Query R101 with benchmark 'Dataset101' and result with BM25 Model for Query 1 

In [51]:
query_benchmark_101_file_path = 'EvaluationBenchmark/Dataset101.txt'
bm25model_r101_result ='My Code/RankingOutputs-New/BM25_R101Ranking.dat' 

In [61]:
if __name__ == "__main__":

    import sys
    import os
    #import coll
    #import df


    # task 2 evaluation
    # get the benchmark
    benFile = open(query_benchmark_101_file_path)
    #benFile = open('Training_benchmark.txt')
    file_ = benFile.readlines()
    ben={}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]]=float(lineList[2])
    benFile.close()
    
    # Print the loaded benchmark data
    #print("Loaded Benchmark Data:")
    #print(ben)
    
    # number documents 
    rank1 = {}
    i = 1
    for line in open(bm25model_r101_result):
        line = line.strip()
        lineList = line.split()
        # Strip '.xml' extension from document IDs
        doc_id = lineList[0].replace('.xml', '')
        rank1[str(i)] = doc_id
        i += 1
        
    #Print the loaded ranked documents
    #print("Loaded Ranked Documents:")
    #print(rank1)

    
    # Evaluation
    print("For task 2:")
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        print("No relevant documents in the benchmark.")
    else:
        for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
            print(f"Evaluating Document ID: {doc_id} at Rank: {n}")
            if doc_id in ben:
                if ben[doc_id] > 0:
                    ri += 1
                    pi = float(ri) / float(int(n))
                    recall = float(ri) / float(R)
                    map1 += pi
                    print(f"At position {int(n)}, precision= {pi}, recall= {recall}")
                else:
                   print(f"Document ID: {doc_id} is found in benchmark but not relevant.")
            else:
                # Instead of printing not found in benchmark for each document, count the not found documents
                print(f"Document ID: {doc_id} is not found in benchmark.")
        if ri > 0:
            map1 = map1 / float(ri)
        else:
            map1 = 0
        print("---The average precision = " + str(map1))

        if ri == 0:
            print("No relevant documents found in the ranked results.")


For task 2:
Evaluating Document ID: 61780 at Rank: 1
Document ID: 61780 is found in benchmark but not relevant.
Evaluating Document ID: 63261 at Rank: 2
At position 2, precision= 0.5, recall= 0.14285714285714285
Evaluating Document ID: 46974 at Rank: 3
At position 3, precision= 0.6666666666666666, recall= 0.2857142857142857
Evaluating Document ID: 46547 at Rank: 4
At position 4, precision= 0.75, recall= 0.42857142857142855
Evaluating Document ID: 22170 at Rank: 5
Document ID: 22170 is found in benchmark but not relevant.
Evaluating Document ID: 62325 at Rank: 6
At position 6, precision= 0.6666666666666666, recall= 0.5714285714285714
Evaluating Document ID: 6146 at Rank: 7
Document ID: 6146 is found in benchmark but not relevant.
Evaluating Document ID: 22513 at Rank: 8
Document ID: 22513 is found in benchmark but not relevant.
Evaluating Document ID: 61329 at Rank: 9
Document ID: 61329 is found in benchmark but not relevant.
Evaluating Document ID: 39496 at Rank: 10
At position 10, pre

### Now Testing code for evaluating model for all the queries benchmark with BM25 model's results for those querries.

For queries 101 to 150 with benchmarks 101...150 for BM25

In [37]:
import os

def load_benchmark(benchmark_file):
    with open(benchmark_file) as benFile:
        file_ = benFile.readlines()
    ben = {}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]] = float(lineList[2])
    return ben

def load_ranked_results(results_file):
    rank = {}
    with open(results_file) as f:
        i = 1
        for line in f:
            line = line.strip()
            lineList = line.split()
            # Strip '.xml' extension from document IDs
            doc_id = lineList[0].replace('.xml', '')
            rank[str(i)] = doc_id
            i += 1
    return rank


def evaluate_query(ben, rank1):
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        return 0, 0, 0  # No relevant documents in the benchmark
    for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
        if doc_id in ben and ben[doc_id] > 0:
            ri += 1
            pi = float(ri) / float(int(n))
            map1 += pi
    if ri > 0:
        map1 = map1 / float(ri)
    else:
        map1 = 0
    recall = float(ri) / float(R) if R > 0 else 0
    return ri, recall, map1

def print_performance_table(performance_data):
    print("\nPerformance Table:")
    print(f"{'Topic':<10} | {'BM25':<10}")
    print("-" * 22)
    for topic, map_value in performance_data:
        print(f"{topic:<10} | {map_value:<10}")

def main():
    query_range = range(101, 151)  # Adjust range as needed
    benchmark_folder = "EvaluationBenchmark"
    results_folder = "My Code/RankingOutputs-New"
    
    total_relevant_items = 0
    total_recall = 0
    total_map = 0
    total_queries = 0

    performance_data = []

    for query_id in query_range:
        benchmark_file = os.path.join(benchmark_folder, f"Dataset{query_id}.txt")
        results_file = os.path.join(results_folder, f"JM_LM_R{query_id}Ranking.dat")
        
        if os.path.exists(benchmark_file) and os.path.exists(results_file):
            ben = load_benchmark(benchmark_file)
            rank1 = load_ranked_results(results_file)
            ri, recall, map1 = evaluate_query(ben, rank1)
            total_relevant_items += ri
            total_recall += recall
            total_map += map1
            total_queries += 1
            print(f"For Query {query_id}: relevant documents (ri)={ri}, recall={recall}, MAP={map1}")
            performance_data.append((f"R{query_id}", map1))
        else:
            print(f"Files for query {query_id} do not exist.")
    
    if total_queries > 0:
        avg_recall = total_recall / total_queries
        avg_map = total_map / total_queries
    else:
        avg_recall = 0
        avg_map = 0

    print(f"\nOverall results for {total_queries} queries:")
    print(f"Total relevant items found: {total_relevant_items}")
    print(f"Average recall: {avg_recall}")
    print(f"Average MAP: {avg_map}")

    # Call the performance table function
    print_performance_table(performance_data)

if __name__ == "__main__":
    main()


For Query 101: relevant documents (ri)=7, recall=1.0, MAP=0.804112554112554
For Query 102: relevant documents (ri)=135, recall=1.0, MAP=0.7014824766514469
For Query 103: relevant documents (ri)=14, recall=1.0, MAP=0.24228592863375
For Query 104: relevant documents (ri)=120, recall=1.0, MAP=0.6379551449146824
For Query 105: relevant documents (ri)=16, recall=1.0, MAP=0.6098968852705267
For Query 106: relevant documents (ri)=4, recall=1.0, MAP=0.3317550505050505
For Query 107: relevant documents (ri)=3, recall=1.0, MAP=0.32083333333333336
For Query 108: relevant documents (ri)=3, recall=1.0, MAP=0.19266381766381765
For Query 109: relevant documents (ri)=20, recall=1.0, MAP=0.656323224421927
For Query 110: relevant documents (ri)=5, recall=1.0, MAP=0.7061302681992337
For Query 111: relevant documents (ri)=3, recall=1.0, MAP=0.10056022408963584
For Query 112: relevant documents (ri)=6, recall=1.0, MAP=0.9583333333333334
For Query 113: relevant documents (ri)=12, recall=1.0, MAP=0.527779900

### Now Testing code for evaluating model for all the queries benchmark with JM_LM model's results for those querries.

In [43]:
import os
import pandas as pd

def load_benchmark(benchmark_file):
    with open(benchmark_file) as benFile:
        file_ = benFile.readlines()
    ben = {}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]] = float(lineList[2])
    return ben

def load_ranked_results(results_file):
    rank = {}
    with open(results_file) as f:
        i = 1
        for line in f:
            line = line.strip()
            lineList = line.split()
            # Strip '.xml' extension from document IDs
            doc_id = lineList[0].replace('.xml', '')
            rank[str(i)] = doc_id
            i += 1
    return rank


def evaluate_query(ben, rank1):
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        return 0, 0, 0  # No relevant documents in the benchmark
    for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
        if doc_id in ben and ben[doc_id] > 0:
            ri += 1
            pi = float(ri) / float(int(n))
            map1 += pi
    if ri > 0:
        map1 = map1 / float(ri)
    else:
        map1 = 0
    recall = float(ri) / float(R) if R > 0 else 0
    return ri, recall, map1

def create_performance_table(performance_data, models):
    df_data = {'Topic': []}
    for model in models:
        df_data[model] = []

    for topic, scores in performance_data.items():
        df_data['Topic'].append(topic)
        for model in models:
            df_data[model].append(scores.get(model, "N/A"))

    df = pd.DataFrame(df_data)
    
    # Calculate the average MAP for each model
    avg_map = {}
    for model in models:
        valid_scores = [score for score in df[model] if score != "N/A"]
        if valid_scores:
            avg_map[model] = sum(valid_scores) / len(valid_scores)
        else:
            avg_map[model] = "N/A"

    # Add a row for the average MAP
    avg_row = {'Topic': 'MAP'}
    for model in models:
        avg_row[model] = avg_map[model]

    df = df.append(avg_row, ignore_index=True)
    
    return df

def main():
    query_range = range(101, 151)  # Adjust range as needed
    benchmark_folder = "EvaluationBenchmark"
    models = {
        "BM25": "My Code/RankingOutputs-New/BM25_R{query_id}Ranking.dat",
        "JM_LM": "My Code/RankingOutputs-New/JM_LM_R{query_id}Ranking.dat",
        "MY_PRM": "Output_Task_3_latest_30_May_arvo/My_PRM_R{query_id}Ranking.dat"
    }
    
    performance_data = {f"R{query_id}": {} for query_id in query_range}

    for model_name, results_pattern in models.items():
        for query_id in query_range:
            benchmark_file = os.path.join(benchmark_folder, f"Dataset{query_id}.txt")
            results_file = results_pattern.format(query_id=query_id)
            
            if os.path.exists(benchmark_file) and os.path.exists(results_file):
                ben = load_benchmark(benchmark_file)
                rank1 = load_ranked_results(results_file)
                ri, recall, map1 = evaluate_query(ben, rank1)
                #print(f"For Query {query_id} ({model_name}): relevant documents (ri)={ri}, recall={recall}, MAP={map1}")
                performance_data[f"R{query_id}"][model_name] = map1
            else:
                #print(f"Files for query {query_id} ({model_name}) do not exist.")
                performance_data[f"R{query_id}"][model_name] = "N/A"

    # Create DataFrame
    performance_df = create_performance_table(performance_data, models.keys())
    print(performance_df.tail())

if __name__ == "__main__":
    main()


   Topic      BM25     JM_LM    MY_PRM
46  R147  0.153922  0.084885  0.124556
47  R148  0.299833  0.392474  0.522388
48  R149  0.215439  0.251429  0.198333
49  R150  0.259524  0.306548  0.385145
50   MAP  0.350229  0.427711  0.315356


/var/folders/6g/w120cf1j2bb4s34_0vnqtdp80000gn/T/ipykernel_15368/1058242543.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(avg_row, ignore_index=True)


### Precision@10

#### Precision@10 = Number of relevant documents in top 10 results by 10

In [47]:
def load_benchmark(benchmark_file):
    """
    Loads benchmark data from the given file.

    Args:
        benchmark_file (str): Path to the benchmark file.

    Returns:
        dict: A dictionary with document IDs as keys and relevance scores as values.
    """
    with open(benchmark_file) as benFile:
        file_ = benFile.readlines()
    ben = {}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]] = float(lineList[2])
    return ben

def load_top_10_results(output_file):
    """
    Loads the top 10 results from the given file.

    Args:
        output_file (str): Path to the file containing the top 10 results.

    Returns:
        dict: A dictionary with ranks as keys and document IDs as values.
    """
    top_10_rank = {}
    with open(output_file) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rank, doc_id = line.split('\t')
            top_10_rank[rank] = doc_id
    return top_10_rank

def evaluate_precision_at_10(ben, top_10_rank):
    """
    Evaluates Precision@10 for the given benchmark data and top 10 results.

    Args:
        ben (dict): Benchmark data with document IDs as keys and relevance scores as values.
        top_10_rank (dict): Top 10 results with ranks as keys and document IDs as values.

    Returns:
        float: Precision@10 score.
    """
    relevant_in_top_10 = 0
    for rank, doc_id in top_10_rank.items():
        if doc_id in ben and ben[doc_id] > 0:
            relevant_in_top_10 += 1
    precision_at_10 = relevant_in_top_10 / 10
    return precision_at_10

def create_precision_at_10_table(query_range, models, benchmark_folder, top_10_results_folder):
    """
    Creates a table with Precision@10 scores for each model and query.

    Args:
        query_range (range): Range of query IDs to evaluate.
        models (dict): Dictionary of models with their result file patterns.
        benchmark_folder (str): Path to the folder containing benchmark files.
        top_10_results_folder (str): Path to the folder containing top 10 results files.

    Returns:
        pandas.DataFrame: DataFrame containing Precision@10 scores for each model and query,
                          along with the average Precision@10 for each model.
    """
    performance_data = {f"R{query_id}": {} for query_id in query_range}

    for model_name in models:
        for query_id in query_range:
            benchmark_file = os.path.join(benchmark_folder, f"Dataset{query_id}.txt")
            top_10_file = os.path.join(top_10_results_folder, f"{model_name}_R{query_id}_Top10.dat")
            
            if os.path.exists(benchmark_file) and os.path.exists(top_10_file):
                ben = load_benchmark(benchmark_file)
                top_10_rank = load_top_10_results(top_10_file)
                precision_at_10 = evaluate_precision_at_10(ben, top_10_rank)
                performance_data[f"R{query_id}"][model_name] = precision_at_10
            else:
                performance_data[f"R{query_id}"][model_name] = "N/A"

    # Create DataFrame
    df_data = {'Topic': []}
    for model in models:
        df_data[model] = []

    for topic, scores in performance_data.items():
        df_data['Topic'].append(topic)
        for model in models:
            df_data[model].append(scores.get(model, "N/A"))

    df = pd.DataFrame(df_data)

    # Calculate the average Precision@10 for each model
    avg_precision_at_10 = {}
    for model in models:
        valid_scores = [score for score in df[model] if score != "N/A"]
        if valid_scores:
            avg_precision_at_10[model] = sum(valid_scores) / len(valid_scores)
        else:
            avg_precision_at_10[model] = "N/A"

    # Add a row for the average Precision@10
    avg_row = {'Topic': 'P@10'}
    for model in models:
        avg_row[model] = avg_precision_at_10[model]

    df = df.append(avg_row, ignore_index=True)
    return df

# Define the query range and models
query_range = range(101, 151)  # Adjust range as needed
benchmark_folder = "EvaluationBenchmark"
top_10_results_folder = "Top10Results"
models = {
    "BM25": "My Code/RankingOutputs-New/BM25_R{query_id}Ranking.dat",
    "JM_LM": "My Code/RankingOutputs-New/JM_LM_R{query_id}Ranking.dat",
    "MY_PRM": "Output_Task_3_latest_30_May_arvo/My_PRM_R{query_id}Ranking.dat"
}

# Create Precision@10 Table
precision_at_10_df = create_precision_at_10_table(query_range, models, benchmark_folder, top_10_results_folder)
print(precision_at_10_df)


   Topic   BM25  JM_LM  MY_PRM
0   R101  0.500  0.500   0.600
1   R102  0.400  0.400   0.500
2   R103  0.200  0.300   0.200
3   R104  0.100  0.500   0.300
4   R105  0.500  0.600   0.400
5   R106  0.200  0.100   0.200
6   R107  0.200  0.200   0.000
7   R108  0.000  0.200   0.000
8   R109  0.200  0.700   0.500
9   R110  0.200  0.400   0.400
10  R111  0.000  0.000   0.000
11  R112  0.400  0.600   0.500
12  R113  0.500  0.500   0.400
13  R114  0.400  0.500   0.300
14  R115  0.200  0.100   0.100
15  R116  0.500  0.000   0.000
16  R117  0.300  0.300   0.200
17  R118  0.300  0.200   0.100
18  R119  0.100  0.100   0.000
19  R120  0.300  0.200   0.300
20  R121  0.300  0.600   0.600
21  R122  0.300  0.500   0.400
22  R123  0.200  0.200   0.000
23  R124  0.100  0.300   0.100
24  R125  0.700  0.500   0.300
25  R126  0.900  0.600   0.800
26  R127  0.400  0.200   0.000
27  R128  0.100  0.100   0.100
28  R129  0.000  0.400   0.100
29  R130  0.300  0.300   0.100
30  R131  0.300  0.200   0.100
31  R132

/var/folders/6g/w120cf1j2bb4s34_0vnqtdp80000gn/T/ipykernel_15368/269471394.py:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(avg_row, ignore_index=True)
